<a href="https://colab.research.google.com/github/zeton24/gsn_iot_anomalies_detection/blob/main/GSN_projekt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Convolutional (Conv2d)
https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d

Average Pooling https://pytorch.org/docs/stable/generated/torch.nn.AvgPool2d.html#torch.nn.AvgPool2d

Normalization
https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html#torch.nn.BatchNorm1d

Spatial Dropout
https://pytorch.org/docs/stable/generated/torch.nn.Dropout2d.html#torch.nn.Dropout2d

Fully connected layer -> nn.Linear()


# Stara wersja

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader

In [ ]:
import torch

## Dataset i Dataloader

todo
 - transformy w datasecie
 - tensory na wyjściu dataloadera

In [ ]:
def to_one_hot(category, n_classes):
  result = torch.zeros(n_classes)
  result[category] = 1
  return result

In [ ]:
# wzięte z https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
# i znacząco zmienione, bo my mamy 1 plik wejściowy, a oni folder z obrazkami
# jak to nie będzie mega powolne, to spoko, ale może być powolne i wtedy nie wiem, co robimy

class IoTDataset(torch.utils.data.Dataset):
  def __init__(self, input_file):
    self.input_file = open(input_file, "r")
    self.dict_of_classes = {"Mirai-Ackflooding": 0, "Mirai-Hostbruteforceg": 1, "Mirai-UDP Flooding": 2, "Mirai-HTTP Flooding": 3,
                            "DoS-Synflooding": 4, "Scan Port OS": 5, "Normal": 6} # todo reszta klas
    self.n_classes = 16 # len(self.dict_of_classes)

  def __len__(self):
    i = 0
    for i, line in enumerate(self.input_file):
      pass
    self.input_file.seek(0)
    return 4 # i + 1

  def __getitem__(self, idx):
    for i, line in enumerate(self.input_file):
      if i == idx:
        data = line.split(";") # docelowo zmienić na ","

        image = torch.Tensor(list(map(float, data[:64])))
        image = torch.reshape(image, (1,64))

        label = data[-1]
        if label.endswith("\n"):
          label = label[:-1]
        label = self.dict_of_classes[label] # -2, lub -3 dla prostszych przypadków
        label = to_one_hot(label, self.n_classes)

        self.input_file.seek(0)
        break
    return image, label

In [ ]:
dataset = IoTDataset("test_dataset.csv")
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [ ]:
for train_features, train_labels in dataloader:
  print(train_features.shape, train_features.dtype)
  print(train_labels)
  break

torch.Size([1, 1, 64]) torch.float32
tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


## Elementy do wykorzystania

### Modele sieci 1D, 2D, 3D

In [ ]:
class Model1D(nn.Module):
    def __init__(self):
        super().__init__()
        self.convblock1 = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=32, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(32),
            nn.AvgPool1d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock2 = nn.Sequential(
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(64),
            nn.AvgPool1d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock3 = nn.Sequential(
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(128),
            nn.AvgPool1d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock4 = nn.Sequential(
            nn.Conv1d(in_channels=128, out_channels=256, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(256),
            nn.AvgPool1d(2),
            nn.Dropout(p=0.05)
        )

        # Nazwa bloku do rozważenia, taka mi się wymyśliła, ale nie upieram się przy niej.
        self.evaluator = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 16)
            # Bez aktywacji na końcu, bo softmax się doda automatycznie razem z cross entropy.
        )

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        x = self.evaluator(x)
        return x

NameError: name 'nn' is not defined

In [ ]:
class Model2D(nn.Module):
    def __init__(self):
        super().__init__()
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            nn.AvgPool2d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.AvgPool2d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.AvgPool2d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),

            # Oni w artykule dają normalizację, ale pytorch wywala błąd.
            # Moim zdaniem słusznie, bo o ile rozumiem ten wzór na normalizację (co jest w dokumentacji),
            # to przy wymiarze wejścia [batch_size, 256, 1, 1] wyjście to tensor zer o takim samym wymiarze.
            # Po prostu normalizacja zmienia nam średnią na 0 (odchylenie standardowe też, ale to nieistotne),
            # a jak mamy 1 element, to zmiana średniej na 0, to zmiana elementu na 0.
            # Więc nawet jak jakoś obejdziemy ten błąd, to wyniki będą bez sensu.
            # Moim zdaniem w artykule jest błąd (a przynajmniej na rysunku, może w implementacji tego nie ma).

            nn.BatchNorm2d(256),
            nn.Dropout(p=0.05)
        )

        self.evaluator = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 16)
        )

    def forward(self, x):
        x = torch.reshape(x, (x.shape[0],1,8,8)) # można ten reshape dać gdzie indziej jak się znajdzie lepsze miejsce
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        x = self.evaluator(x)
        return x

In [ ]:
class Model3D(nn.Module):
    def __init__(self):
        super().__init__()
        self.convblock1 = nn.Sequential(
            nn.Conv3d(in_channels=1, out_channels=32, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm3d(32),
            nn.AvgPool3d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock2 = nn.Sequential(
            nn.Conv3d(in_channels=32, out_channels=64, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm3d(64),
            nn.AvgPool3d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock3 = nn.Sequential(
            nn.Conv3d(in_channels=64, out_channels=128, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm3d(128),
            nn.Dropout(p=0.05)
        )

        self.convblock4 = nn.Sequential(
            nn.Conv3d(in_channels=128, out_channels=256, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm3d(256),
            nn.Dropout(p=0.05)
        )

        self.evaluator = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 16)
        )

    def forward(self, x):
        x = torch.reshape(x, (x.shape[0],1,4,4,4))
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        x = self.evaluator(x)
        return x

### Batch loader

In [ ]:
# Na bazie datasetu z labów - zwraca kolejne batche.
# Nie jest jakiś genialny, ale i tak go używam tylko tu do testów.

class Dataset:
  def __init__(self, data, labels, batch_size=1):
    self.data = data
    self.labels = labels
    self.batch_size = batch_size
    self.n_batches = len(self.data) // batch_size

  def __iter__(self):
    for i in range(self.n_batches):
      inputs = torch.zeros(self.batch_size, 1, 64) # shape zahardkodowany
      labels = torch.zeros(self.batch_size, self.labels.shape[1])

      for j in range(self.batch_size):
        inputs[j, 0] = self.data[i*self.batch_size+j]
        labels[j]= self.labels[i*self.batch_size+j]

      yield inputs, labels

  # todo - jakiś shuffle by się przydał, ale to tylko jeśli byśmy z tego korzystały

### Funkcje do treningu i testowania

In [ ]:
# Funkcja przeprowadzająca traning
def train(net, loader, optimizer, criterion, epochs):
  net.train()
  for epoch in range(epochs):
    epoch_loss = 0.
    for i, (inputs, labels) in enumerate(loader):

      # zero the parameter gradients
      optimizer.zero_grad()

      outputs = net(inputs)
      loss = criterion(outputs, labels)

      loss.backward()
      optimizer.step()

      epoch_loss += loss

    # Wczesne przerwanie w razie dostatecznie dobrych wyników.
    if epoch_loss<0.05:
      print(f'[{epoch + 1}, {i + 1:5d}] eloss: {epoch_loss:.3f}')
      break

    if epoch % 20 == 19: # wypisywanie co 20 epok
      print(f'[{epoch + 1}, {i + 1:5d}] eloss: {epoch_loss:.3f}')

In [ ]:
def test(network, test_set, n_classes = 16):
  network.eval() # Przełącza sieć w tryb testowania, m. in. wyłącza dropouta.
  err_matrix = torch.zeros((n_classes, n_classes), dtype=int)

  with torch.no_grad():

    for inputs, labels in test_set:

      outputs = network(inputs)

      _, label = torch.max(labels, 1)
      _, predicted = torch.max(outputs, 1)

      for truth, prediction in zip(label, predicted):
        err_matrix[truth, prediction] += 1

  return err_matrix

In [ ]:
# Funkcja tworzy losowe etykiety one-hot.

def rand_and_predict(dataset_size, output_length):
  labels = torch.zeros(dataset_size, output_length)
  for i in range(dataset_size):
    hot = torch.randint(output_length,(1,))
    labels[i, hot] = 1
  return labels


## Test na małych danych

In [ ]:
inputsR = torch.rand(24,1,64)
labelsR = rand_and_predict(24,16)

In [ ]:
classes = {"Mirai-Ackflooding": 0, "Mirai-Hostbruteforceg": 1, "Mirai-UDP Flooding": 2, "Mirai-HTTP Flooding": 3, "DoS-Synflooding": 4, "Scan Port OS": 5, "Normal": 6}
inputs = torch.zeros(24,1,64)
labels = torch.zeros(24,16)
with open('test_dataset.csv') as f:
  for i, line in enumerate(f):
    data = line.split(";") # docelowo zmienić na ","
    image = torch.Tensor(list(map(float, data[1:65])))
    image = torch.reshape(image, (1,64))

    label = data[-1]
    if label.endswith("\n"):
      label = label[:-1]
    label = classes[label]
    label = to_one_hot(label, 16)

    inputs[i] = image
    labels[i] = label

In [ ]:
loader = Dataset(inputs, labels, batch_size=1)

In [ ]:
criterion = nn.CrossEntropyLoss()
criterion_test = nn.CrossEntropyLoss()

### Model 1D

In [ ]:
network1 = Model1D()
optimizer1 = optim.Adam(network1.parameters(), lr=0.0001)

In [ ]:
train(network1, dataloader, optimizer1, criterion, 1000)

[20,     2] eloss: 0.773
[40,     2] eloss: 0.464
[60,     2] eloss: 0.220
[65,     2] eloss: 0.046


In [ ]:
result1 = test(network1, dataloader)
print(f"Accuracy: {sum(sum(result1*torch.eye(16)))*100/sum(sum(result1)):.2f}%")
print(result1)

Accuracy: 100.00%
tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


### Model 2D

In [ ]:
network2 = Model2D()
optimizer2 = optim.Adam(network2.parameters(), lr=0.001)

In [ ]:
train(network2, dataloader, optimizer2, criterion, 100)

[20,     2] eloss: 3.156
[40,     2] eloss: 3.729
[60,     2] eloss: 2.365
[80,     2] eloss: 1.382
[100,     2] eloss: 0.487


In [ ]:
result2 = test(network2, dataloader)
print(f"Accuracy: {sum(sum(result2*torch.eye(16)))*100/sum(sum(result2)):.2f}%")
print(result2)

Accuracy: 75.00%
tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


### Model 3D

In [ ]:
network3 = Model3D()
optimizer3 = optim.Adam(network3.parameters(), lr=0.001)

In [ ]:
train(network3, dataloader, optimizer3, criterion, 100)

[20,     2] eloss: 1.973
[40,     2] eloss: 1.932
[60,     2] eloss: 2.405
[80,     2] eloss: 3.536
[100,     2] eloss: 2.386


In [ ]:
result3 = test(network3, dataloader)
print(f"Accuracy: {sum(sum(result3*torch.eye(16)))*100/sum(sum(result3)):.2f}%")
print(result3)

Accuracy: 50.00%
tensor([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


# Lightning

In [1]:
!pip install polars --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 13.0 MB/s eta 0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 0.20.2
    Uninstalling polars-0.20.2:
      Successfully uninstalled polars-0.20.2


In [2]:
! pip install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.2/802.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 26.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-many

In [3]:
import os
import gdown
import torch
import pytorch_lightning as pl
import polars
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
from torchmetrics import Accuracy
import torch.nn.functional as F

In [ ]:
#import PIL
#import torchmetrics
#from IPython.display import display
#from pytorch_lightning import Trainer
#from scipy.io import loadmat

#from torchvision import models
#from torchvision.datasets import ImageFolder
#from torchvision.transforms import Resize, Compose, ToTensor, Normalize, ToPILImage
#import hydra
#from hydra.utils import instantiate

#import torchvision
#from torchmetrics.classification import accuracy


### dane

In [4]:
class LazyDataset(torch.utils.data.Dataset):
  def __init__(self, inputs: torch.tensor, labels: torch.tensor, n_classes: int):
    self.inputs = inputs
    self.labels = labels
    self.n_classes = n_classes

  def __len__(self):
    return self.labels.shape[0]

  def __getitem__(self, idx):
    temp = torch.zeros(self.n_classes)
    temp[self.labels[idx].item()] = 1
    return self.inputs[idx], temp

In [5]:
class IoTDataModule(pl.LightningDataModule):
  def __init__(self, file_id: str, file_name: str, batch_size: int=64, binary_classification: bool=False):
    super().__init__()
    self.file_id = file_id
    self.file_name = file_name
    self.batch_size = batch_size
    self.binary_classification = binary_classification

    if self.binary_classification:
      self.n_classes = 2
      self.mapping = {"Normal": 0, "Anomaly": 1}
    else:
      self.n_classes = 5
      self.mapping = {'Normal': 0, 'Mirai': 1, 'DoS': 2, 'Scan': 3, 'MITM ARP Spoofing': 4}

  def prepare_data(self):
    if not os.path.exists(self.file_name):
      gdown.download(id=self.file_id, output=self.file_name)

  def setup(self,stage = None):
    data = torch.reshape(polars.read_csv(self.file_name, columns = range(64)).cast(polars.Float32).to_torch(), (-1, 1, 64))

    if self.binary_classification:
      labels = torch.reshape(polars.read_csv(self.file_name, columns = [64])['Label'].replace(self.mapping, return_dtype=polars.UInt8).to_torch(), (-1, 1))
    else:
      #labels = torch.reshape(polars.read_csv(self.file_name, columns = [65])['Cat'].replace(self.mapping, return_dtype=polars.UInt8).to_torch(), (-1, 1))
      labels = polars.read_csv(self.file_name, columns = [65])['Cat'].replace(self.mapping, return_dtype=polars.UInt8).to_torch()
    dataset = LazyDataset(data, labels, self.n_classes)
    del data, labels

    # Train, test i val.
    l = len(dataset)
    train_and_val_size = int(l * .75)
    dataset, self.test_dataset = random_split(dataset, [train_and_val_size, l - train_and_val_size])

    ll = len(dataset)
    train_size = int( ll * .9)
    self.train_dataset, self.val_dataset = random_split(dataset, [train_size, ll - train_size])
    del dataset

  def train_dataloader(self):
    return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

  def val_dataloader(self):
    return DataLoader(self.val_dataset, batch_size=self.batch_size)

  def test_dataloader(self):
    return DataLoader(self.test_dataset, batch_size=self.batch_size)

In [6]:
dm = IoTDataModule(file_id='1k8RqOM7hBvL8uomcKnnLr5HasYRWnYeW', file_name='iot-intrusion_with_headers.csv')
dm.prepare_data()
dm.setup()

Downloading...
From (original): https://drive.google.com/uc?id=1k8RqOM7hBvL8uomcKnnLr5HasYRWnYeW
From (redirected): https://drive.google.com/uc?id=1k8RqOM7hBvL8uomcKnnLr5HasYRWnYeW&confirm=t&uuid=865e1931-e4b9-4c1c-ae38-b8570b1f7841
To: /content/iot-intrusion_with_headers.csv
100%|██████████| 212M/212M [00:03<00:00, 56.8MB/s]


In [105]:
channels_sum, channels_squared_sum, num_batches = 0, 0, 0
loader = dm.val_dataloader()
for data, _ in loader:
    if data.isinf().any():
      print(data.isinf().sum())
    channels_sum += torch.mean(data, dim=[0,2])
    channels_squared_sum += torch.mean(data**2, dim=[0, 2])
    num_batches += 1

print('bacthes', num_batches)
print('channel sum', channels_sum, channels_sum == torch.inf)

print('channels_squared_sum', channels_squared_sum)
mean = channels_sum/num_batches
std = (channels_squared_sum/num_batches - mean**2)**0.5
print(mean, std)

bacthes 733
channel sum tensor([1.4440e+08]) tensor([False])
channels_squared_sum tensor([3.5392e+15])
tensor([196994.4844]) tensor([2188503.2500])


In [23]:
a = len(dm.train_dataset)
b = len(dm.val_dataset)
c = len(dm.test_dataset)
d=a+b+c

In [25]:
print(d, a)
print(a/d*40/27)
print(b/d*40/3)
print(c/d*4)

625783 422403
0.9999987571126449
1.000005859326103
1.0000015979980281


In [34]:
for input, label in dm.train_dataloader():
  isnan = input.isnan().sum().item()
  if isnan != 0:
    print(input)
  #print(label.T)

### model

In [7]:
class Model1D(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.convblock1 = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=32, kernel_size=5, padding='same'),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.AvgPool1d(2),
            nn.Dropout(p=0.05)
        )

        self.c1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=5, padding='same')
        self.relu1 = nn.ReLU()
        self.norm1 = nn.BatchNorm1d(32)
        self.pool1 = nn.AvgPool1d(2)
        self.drop1 = nn.Dropout(p=0.05)

        self.convblock2 = nn.Sequential(
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(64),
            nn.AvgPool1d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock3 = nn.Sequential(
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(128),
            nn.AvgPool1d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock4 = nn.Sequential(
            nn.Conv1d(in_channels=128, out_channels=256, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(256),
            nn.AvgPool1d(2),
            nn.Dropout(p=0.05)
        )

        # Nazwa bloku do rozważenia, taka mi się wymyśliła, ale nie upieram się przy niej.
        self.evaluator = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, num_classes)
            # Bez aktywacji na końcu, bo softmax się doda automatycznie razem z cross entropy.
        )

    def forward(self, x):
      if x.isnan().any():
        print(x.isnan().sum())
        raise Exception('layaer wtf')
      x = self.c1(x)
      if x.isnan().any():
        print(x.isnan().sum())
        print(x)
        raise Exception('layaer 00')
      x = self.relu1(x)
      if x.isnan().any():
        print(x.isnan().sum())
        raise Exception('layaer 01')
      x = self.norm1(x)
      if x.isnan().any():
        print(x.isnan().sum())
        raise Exception('layaer 02')
      x = self.pool1(x)
      if x.isnan().any():
        print(x.isnan().sum())
        raise Exception('layaer 03')
      x = self.drop1(x)
      #x = self.convblock1(x)
      if x.isnan().any():
        print(x.isnan().sum())
        raise Exception('layaer 1')
      x = self.convblock2(x)
      if x.isnan().any():
        raise Exception('layaer 22')
      x = self.convblock3(x)
      if x.isnan().any():
        raise Exception('layaer 3')
      x = self.convblock4(x)
      if x.isnan().any():
        raise Exception('layaer 4')
      x = self.evaluator(x)
      return x

In [8]:
class AnomalyClassifier(pl.LightningModule):

  def __init__(self, lr, model_type=1, binary_classification=False):
    super().__init__()
    self.lr = lr
    self.current_epoch_training_loss = torch.tensor(0.0)
    self.training_step_outputs = []
    self.validation_step_outputs = []

    self.num_classes = 2 if binary_classification else 5
    self.accuracy = Accuracy(task='multiclass', num_classes=self.num_classes) # I sposób na metryke

    if model_type == 1:
      self.model = Model1D(self.num_classes)
    elif model_type == 2:
      self.model = Model2D(self.num_classes)
    else:
      self.model = Model3D(self.num_classes)

  def forward(self, x):
    return self.model(x)

  def compute_loss(self, x, y):
    #print(x)
    return F.cross_entropy(x, y)

  def common_step(self, batch, batch_idx):
    x, y = batch
    outputs = self(x)
    loss = self.compute_loss(outputs,y)
    return loss, outputs, y

  def common_test_valid_step(self, batch, batch_idx):
    loss, outputs, y = self.common_step(batch, batch_idx)
    preds = torch.argmax(outputs, dim=1)
    acc = self.accuracy(outputs, y)
    return loss, acc

  def training_step(self, batch, batch_idx):
    loss, outputs, y = self.common_step(batch, batch_idx)
    self.training_step_outputs.append(loss)
    accuracy = self.accuracy(outputs, y)
    print(f'train_loss: {loss}, train_acc: {accuracy}')
    self.log_dict({"train_loss": loss, "train_accuracy": accuracy}, on_step = False, on_epoch = True, prog_bar = True) # logger=True?
    return {'loss':loss}

  def on_train_epoch_end(self):
    outs = torch.stack(self.training_step_outputs)
    self.current_epoch_training_loss = outs.mean()
    print(f'train loss: {self.current_epoch_training_loss}')
    self.training_step_outputs.clear()

  def validation_step(self, batch, batch_idx):
    loss, acc = self.common_test_valid_step(batch, batch_idx)
    self.validation_step_outputs.append(loss)

    self.log_dict({'val_loss': loss, 'val_acc': acc}, on_step=True, on_epoch=True, prog_bar=True, logger=True)
    print(f'val_loss: {loss}, val_acc: {acc}')
    return {'val_loss':loss, 'val_acc': acc}

  def on_validation_epoch_end(self):
    outs = torch.stack(self.validation_step_outputs)
    avg_loss = outs.mean()
    self.logger.experiment.add_scalars('train and vall losses', {'train': self.current_epoch_training_loss.item() , 'val': avg_loss.item()}, self.current_epoch)
    self.validation_step_outputs.clear()

  def test_step(self, batch, batch_idx):
    loss, acc = self.common_test_valid_step(batch, batch_idx)

    self.log_dict({'test_loss': loss, 'test_acc': acc}, on_step=True, on_epoch=True, prog_bar=True, logger=True)
    return {'test_loss': loss, 'test_acc': acc}

  def configure_optimizers(self):
    optimizer =  torch.optim.Adam(self.parameters(), lr=self.lr)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
    return [optimizer], [lr_scheduler]

### trening

In [9]:
classifier = AnomalyClassifier(0.001, model_type=1)
trainer = pl.Trainer(accelerator="auto", max_epochs=10, precision=32)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [10]:
trainer.fit(classifier, dm)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name     | Type               | Params
------------------------------------------------
0 | accuracy | MulticlassAccuracy | 0     
1 | model    | Model1D            | 744 K 
------------------------------------------------
744 K     Trainable params
0         Non-trainable params
744 K     Total params
2.977     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

val_loss: 5066.646484375, val_acc: 0.0
val_loss: 4734.69921875, val_acc: 0.0


Training: |          | 0/? [00:00<?, ?it/s]

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
train_loss: 0.4647338092327118, train_acc: 0.0
train_loss: 0.5117170214653015, train_acc: 0.0
train_loss: 0.5695328712463379, train_acc: 0.0
train_loss: 0.37894049286842346, train_acc: 0.0
train_loss: 0.48507264256477356, train_acc: 0.0
train_loss: 0.4350612163543701, train_acc: 0.0
train_loss: 0.6283729076385498, train_acc: 0.0
train_loss: 0.31245937943458557, train_acc: 0.0
train_loss: 0.6057425141334534, train_acc: 0.0
train_loss: 0.46291401982307434, train_acc: 0.0
train_loss: 0.7654971480369568, train_acc: 0.0
train_loss: 0.44052112102508545, train_acc: 0.0
train_loss: 0.5732763409614563, train_acc: 0.0
train_loss: 0.6960064172744751, train_acc: 0.0
train_loss: 0.521155834197998, train_acc: 0.0
train_loss: 0.47366949915885925, train_acc: 0.0
train_loss: 0.4501166045665741, train_acc: 0.0
train_loss: 0.44767019152641296, train_acc: 0.0
train_loss: 0.7191113233566284, train_acc: 0.0
train_loss: 0.559852123260498, train

...

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
train_loss: 0.3974573314189911, train_acc: 0.0
train_loss: 0.3408347964286804, train_acc: 0.0
train_loss: 0.34198522567749023, train_acc: 0.0
train_loss: 0.5258795022964478, train_acc: 0.0
train_loss: 0.2219962626695633, train_acc: 0.0
train_loss: 0.4405249357223511, train_acc: 0.0
train_loss: 0.40458616614341736, train_acc: 0.0
train_loss: 0.43515682220458984, train_acc: 0.0
train_loss: 0.3805353343486786, train_acc: 0.0
train_loss: 0.41849958896636963, train_acc: 0.0
train_loss: 0.16709575057029724, train_acc: 0.0
train_loss: 0.34365037083625793, train_acc: 0.0
train_loss: 0.2218005359172821, train_acc: 0.0
train_loss: 0.40537577867507935, train_acc: 0.0
train_loss: 0.33541080355644226, train_acc: 0.0
train_loss: 0.3548007309436798, train_acc: 0.0
train_loss: 0.22244000434875488, train_acc: 0.0
train_loss: 0.5441092252731323, train_acc: 0.0
train_loss: 0.4471461772918701, train_acc: 0.0
train_loss: 0.26635801792144775, 

Validation: |          | 0/? [00:00<?, ?it/s]

val_loss: 0.38744914531707764, val_acc: 0.0
val_loss: 0.38284027576446533, val_acc: 0.0
val_loss: 0.47272276878356934, val_acc: 0.0
val_loss: 0.5328455567359924, val_acc: 0.0
val_loss: 0.34890449047088623, val_acc: 0.0
val_loss: 0.5891264081001282, val_acc: 0.0
val_loss: 0.43218994140625, val_acc: 0.0
val_loss: 0.585624098777771, val_acc: 0.0
val_loss: 0.6104098558425903, val_acc: 0.0
val_loss: 0.43738067150115967, val_acc: 0.0
val_loss: 0.4770536422729492, val_acc: 0.0
val_loss: 0.3780099153518677, val_acc: 0.0
val_loss: 0.6782466173171997, val_acc: 0.0
val_loss: 0.5983327031135559, val_acc: 0.0
val_loss: 0.395277202129364, val_acc: 0.0
val_loss: 0.6431982517242432, val_acc: 0.0
val_loss: 0.6013569831848145, val_acc: 0.0
val_loss: 0.39934709668159485, val_acc: 0.0
val_loss: 0.5992019176483154, val_acc: 0.0
val_loss: 0.7544083595275879, val_acc: 0.0
val_loss: 0.4823288917541504, val_acc: 0.0
val_loss: 0.46723952889442444, val_acc: 0.0
val_loss: 0.35750719904899597, val_acc: 0.0
val_los

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


train loss: 0.3705449402332306


# Poligon

Tu testuję różne rzeczy, ale tak, żeby się nie mieszały z resztą

In [ ]:
from sys import getsizeof

In [ ]:
import numpy as np
import pandas as pd
import torch

In [ ]:
file_path = "x.csv"
arr_pan = pd.read_csv(file_path, header=None, sep=";").astype(np.float16)
arr_pol = pol.read_csv(file_path, has_header=False, separator=";").cast(pol.Float32)
arr_num = arr_pan.to_numpy()
print(arr_pol.dtypes)
print(arr_num.dtype)
arr_tor = torch.from_numpy(arr_num)
print(arr_tor.dtype)


[Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32]
float16
torch.float16


/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/astype.py:138: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)


In [ ]:
print(arr_pan.memory_usage()[0]*64+arr_pan.memory_usage()['Index'])
print(arr_pol.estimated_size())
print(arr_num.nbytes)
print(arr_tor.storage().nbytes())

3200
6144
3072
3072


In [ ]:
print(type(arr_pan[0][0]))

<class 'numpy.int64'>


In [ ]:
print(arr_pol)

shape: (120, 64)
┌──────────┬──────────┬──────────┬──────────┬───┬───────────┬─────────────┬───────────┬───────────┐
│ column_1 ┆ column_2 ┆ column_3 ┆ column_4 ┆ … ┆ column_61 ┆ column_62   ┆ column_63 ┆ column_64 │
│ ---      ┆ ---      ┆ ---      ┆ ---      ┆   ┆ ---       ┆ ---         ┆ ---       ┆ ---       │
│ i64      ┆ i64      ┆ i64      ┆ f64      ┆   ┆ str       ┆ f64         ┆ f64       ┆ f64       │
╞══════════╪══════════╪══════════╪══════════╪═══╪═══════════╪═════════════╪═══════════╪═══════════╡
│ 75       ┆ 1        ┆ 1        ┆ 982.0    ┆ … ┆ 75.0      ┆ 0.0         ┆ 75.0      ┆ 75.0      │
│ 5310     ┆ 1        ┆ 2        ┆ 0.0      ┆ … ┆ 2655.0    ┆ 2261.327486 ┆ 4254.0    ┆ 1056.0    │
│ 141      ┆ 0        ┆ 3        ┆ 0.0      ┆ … ┆ 70.5      ┆ 0.707107    ┆ 71.0      ┆ 70.0      │
│ 151      ┆ 0        ┆ 2        ┆ 0.0      ┆ … ┆ 151.0     ┆ 0.0         ┆ 151.0     ┆ 151.0     │
│ …        ┆ …        ┆ …        ┆ …        ┆ … ┆ …         ┆ …           ┆ …      

In [ ]:
print(arr_num)

[[75 1 1 ... 0.0 75.0 75.0]
 [5310 1 2 ... 2261.327486234579 4254.0 1056.0]
 [141 0 3 ... 0.7071067811865476 71.0 70.0]
 ...
 [74 1 1 ... 0.0 74.0 74.0]
 [123 0 2 ... 0.0 123.0 123.0]
 [199 1 1 ... 0.0 199.0 199.0]]


In [ ]:
print(arr_tor)

In [ ]:
s = torch.tensor([[1,2], [3,4]])
print(s)

tensor([[1, 2],
        [3, 4]])


In [ ]:
t = torch.reshape(s, (-1, 4))
u = torch.reshape(s, (-1, 1))

In [ ]:
print(t)
print(u)

tensor([[1, 2, 3, 4]])
tensor([[1],
        [2],
        [3],
        [4]])


# Notatki, żeby nie szukać ciągle w artykule, bałagan trochę, ale są

koszt - cross entropy

aktywacja lineara - relu

aktywacja po 16 neuronach - softmax

zrobili L1, L2 i dropout, ale nie wchodziłam w to dokładnie

batch - 64, 128 dawały najlepsze wyniki

eksperymentalnie sprawdzili, że 100 epok daje zbieżność

w adamie dali 0.0001 learning rate
